# Grocery Store Suggestions System

---




In the daily life, we use to forget something and realize after arriving at home. To solve this problem and increase the client satisfaction, our team suggest a recommendation system for online and off-line store in this project. We aim to help our customer shop in a convenient environment by suggesting the related producted based on the actual data presendted by bonÀrea. 

## 1. Business use case

### Goals

In this project, 
The objective of this project is to do an analysis of the sales history of the stores and purchase data to predict products that are highly relevant to the purchasing products. 
Based on this prediction, we present a recommendation solution considering purchasing circumstance like holiday, event, or time. 

The different applications of this analysis can be suggestions on the website at the moment of the payment, on the waiting lines, printed on the receipt (as a reminder for the next time).

These solutions can be helpful to provide customer-oriented experience to our clients. We are able to expect the increase of the potencial customer and long-term sales performance. 


### Possible Obstacles

- The computation with big datasets could be  higher an do it for each store (we  have more than 500) could take a long time.
- The results need to be used in the low performance computers as POS (Point of Sale) systems.
- Determine in advance the price of the computation.
- In which way is it possible to combine dataset related to the purchasing habits of different socio-economic groups with those we have. It could add some complexity to our analysis.
- Short time to do the project.


## 2. Technical requirements

### Data sources

The company of Grocery stores has a headquarter where all information of stores are collected to its databases. From these databases with ETLs the data is exported to the csv files  that are needed for this project.  (These part is done in company and is out of range of this document).

The files are 3: 
- The **families** 
- The **products**
- The **breakdown of receipts**

The lines could be related with families by products table.








#### Families:

The families are the form to group products by similar types. Each scale of group the similarity are higher. This families are based from AECOC offers (https://www.aecoc.es/guias/clasificacion-estandarizada-de-producto/).

AECOC is an Spanish association of producers and distributors that is also associated with a global GS1 organization. One of their targets is offer a several standards and family codes is one of them.

The standard offers till 5 levels of classification but this company nowadays only works with 3 levels.

Sample of some lines:
```csv
FAMILIA ;DESCRIPCIO.SECTOR ; DESCRIPCIO.SECCIO  ;DESC.FAMILIA ;;;;
01*01*01 ;Alimentacion y Bebidas  ;ALIMENTACIÓN SECA ;Aceites ;;;;
01*01*02 ;Alimentacion y Bebidas  ;ALIMENTACIÓN SECA ;Cafés y sucedáneos ;;;;
...
01*02*01 ;Alimentacion y Bebidas  ;CONSERVAS ;Conservas de pescado y marisco ;;;;
01*02*02 ;Alimentacion y Bebidas  ;CONSERVAS ;Conservas vegetales ;;;;
...
01*03*01 ;Alimentacion y Bebidas  ;Làcteos y derivados ;Leche ;;;;
01*03*02 ;Alimentacion y Bebidas  ;Làcteos y derivados ;Leches no liquidas ;;;;
...
01*04*01 ;Alimentacion y Bebidas  ;BEBIDAS ;Aguas ;;;;
01*04*02 ;Alimentacion y Bebidas  ;BEBIDAS ;Bebidas refrescantes ;;;;
...
```

#### Products:
This file comes with a properly headers on csv file. This file relate each product with its family. The information of family is joined in a unique string and splited by sector,section and family.

Sample of some lines:
```csv
ARTICLE;DESCRIPCIO;SECTOR;SECCIO;FAMILIA;DESC
10002;LIMPIACRISTALES BONA;3;1;6;03*01*06
10003;LIMPIACRISTALES BONA;3;1;6;03*01*06
10004;FREGASUELOS CAG 1 L;3;1;6;03*01*06
10006;VAJILLAS CONCENTRADO;3;1;4;03*01*04
10007;VAJILLAS VERDE BONAC;3;1;4;03*01*04
...
```

#### Breakdown of receipt:

This file contains each product that are sold. The invoice_id group each line that is sold to unique customer. Each file contains all lines sold by one checkout in a year. So, This could have easily more that one milion of lines.

This file comes without headers. But we obtained the meaning of each column.

1. Invoice id. This code inform us that all lines with same id are sold to the same customer. Also has a correlated number and where is done (store number and checkout number). The format are:
 - 2 digit: year
 - 2 digit: company department
 - dash '-'
 - T + 4 digits : Store number
 - C + 2 digits : Checkout number
 - dash '-'
 - Number of invoice.
2. Product id.
3. Units. This column is multiplied by one million.
4. Amount. This is total amount that the customer pay, not the product price. This is multiplied by a hundred.
5. Day. Day in Julian format that Julian period starts on January 1th, 1968. Also with modulus of 7 we can know the week day (0 = Sunday).
7. Time. Is the number of seconds from midnight.

Sample of some lines:
```csv
2027-T0101C01-100089;3055;1000000;24;01;19343;35707
2027-T0101C01-100089;3055;1000000;24;01;19343;35707
2027-T0101C01-100089;6989;1000000;0;01;19343;35707
2027-T0101C01-100188;8939;1000000;226;01;19343;44143
2027-T0101C01-100188;8939;1000000;228;01;19343;44143
...
```

### How to work with the data

The Data will be explored using techniques of Artificial intelligence. A-priori style algorithms or using some kind of Neural Networks. 

Some kinds of classification could do that the results are different. For example according to the time, if the day is holiday or the time of year. The total amount or the number of products. etc. 

The brand of product should not be important so other kind of job is work with the families at distinct levels. On the data exploration, we observed that some families have tens of products, so we decided to work with products.

The computation is convenient to be with some architecture that allows to parallelize the jobs. The number of stores and sells are big and it's desired that the time to get results was short. So, use utilities as spark could be a good ally.

The place to work at the beginning is *Google Colab®* because they offer us a shared platform to work collaboratively and computers with GPU or TPU deliver something to carry out the work. When the project evolves it is possible to hire more or more large services.

This project has just started in *Google Colab®* and it has started with open and explore data from csv files.

As *Google Collab®* doen't has Spark installed by default, next block prepare the environment to can work with Spark. Also as this platform is efimer the data is managed with a mounted volume to *Google Drive®*. 

In [ ]:
try:
  # Only it runs in google collab
  from google.colab import drive # if this package don't exist we aren't con google colab
  import os
  # Install spark and its requirements
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  !wget -nc -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
  !tar xf spark-3.2.0-bin-hadoop3.2.tgz
  !pip install -q pyspark
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
  drive.mount('/content/drive')
  base_path = '/content/drive/MyDrive/Master/Project Big Data/Project Big Data/' # Google Drive
except:
  # It runs in local 
  base_path = ''


     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 69.1 MB/s 
Mounted at /content/drive


Importing libraries to work with Spark and initializate a SparkSession.

In [ ]:
# Import Spark libs
from pyspark.sql import SparkSession
from IPython.display import display, HTML
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col


# Initiate SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .getOrCreate()

The path of files. This uses a base path in order to be able to work this Jupyter notebook wherever (at moment in colab and local).

In [ ]:
# File Path
file_products = base_path + 'data/ARTICLES.csv'
file_families = base_path + 'data/AECOC.csv'
file_sells = base_path + 'data/s20.csv'

#### Products file

In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

df_product = spark.read.format('csv') \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_products)
df_product.limit(5).toPandas()

,ARTICLE,DESCRIPCIO,SECTOR,SECCIO,FAMILIA,DESC
0,10000,LIMPIACRISTALES CON,3,1,6,03*01*06
1,10001,LIMPIACRISTALES RECA,3,1,6,03*01*06
2,10002,LIMPIACRISTALES BONA,3,1,6,03*01*06
3,10003,LIMPIACRISTALES BONA,3,1,6,03*01*06
4,10004,FREGASUELOS CAG 1 L,3,1,6,03*01*06


It make a projection of this file with selected columns and normalized its name.

In [ ]:
print ("total products: ", df_product.count())
df_product.selectExpr('ARTICLE as product_id','DESC as family_id', 'DESCRIPCIO as product_desc' ).limit(5).toPandas()

total products:  72200


,product_id,family_id,product_desc
0,10000,03*01*06,LIMPIACRISTALES CON
1,10001,03*01*06,LIMPIACRISTALES RECA
2,10002,03*01*06,LIMPIACRISTALES BONA
3,10003,03*01*06,LIMPIACRISTALES BONA
4,10004,03*01*06,FREGASUELOS CAG 1 L


#### Families file

In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

df_families = spark.read.format('csv') \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_families)

df_families.limit(5).toPandas()

,FAMILIA,DESCRIPCIO.SECTOR,DESCRIPCIO.SECCIO,DESC.FAMILIA,_c4,_c5,_c6,_c7
0,00*00*00,Desconocido,,Desconeguda,None,None,None,None
1,01*01*01,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Aceites,None,None,None,None
2,01*01*02,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Cafés y sucedáneos,None,None,None,None
3,01*01*03,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Infusiones,None,None,None,None
4,01*01*04,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Chocolates,None,None,None,None


In [ ]:
schema = StructType([
    StructField("family_id", StringType(), True),
    StructField("sector_desc", StringType(), True),
    StructField("section_desc", StringType(), True),
    StructField("family_desc", StringType(), True),
    StructField("not_use1", StringType(), True),
    StructField("not_use2", StringType(), True),
    StructField("not_use3", StringType(), True),
    StructField("not_use4", StringType(), True)
    ])

df_families = spark.read.format('csv') \
  .schema(schema) \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_families)


print ("total families: ", df_families.count())

df_families.select(col('family_id'), col('family_desc') ).limit(5).toPandas()


total families:  816


,family_id,family_desc
0,00*00*00,Desconeguda
1,01*01*01,Aceites
2,01*01*02,Cafés y sucedáneos
3,01*01*03,Infusiones
4,01*01*04,Chocolates


In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ";"

schema = StructType([
    StructField("invoice_id", StringType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("amount", IntegerType(), True),
    StructField("checkout", IntegerType(), True),
    StructField("date", IntegerType(), True),
    StructField("hour", IntegerType(), True),])

df_sells = spark.read.format('csv') \
  .schema(schema) \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_sells)

df_sells.limit(5).toPandas()


,invoice_id,product_id,quantity,amount,checkout,date,hour
0,2027-T0105C01-100089,5379,1000000,171,1,19282,64396
1,2027-T0105C01-100089,5379,1000000,160,1,19282,64396
2,2027-T0105C01-100089,3482,1000000,63,1,19282,64396
3,2027-T0105C01-100089,3059,1000000,45,1,19282,64396
4,2027-T0105C01-100089,3059,1000000,45,1,19282,64396


In [ ]:
from datetime import datetime,timedelta

print("total sells: ", df_sells.count())

StartDate = "12/31/1967"
Date = datetime.strptime(StartDate, "%m/%d/%Y")

df_sells = df_sells.rdd \
  .map(lambda x: (x[0],x[1],x[2]/1000000,x[3]/100,x[4],Date.replace(hour=0, minute=0, second=0, microsecond =0)+timedelta(days=x[5], seconds=x[6]))) \
  .toDF(['invoice_id','product_id', 'quantity','amount','checkout','datetime'])

df_sells.limit(5).toPandas()

total sells:  1263376


,invoice_id,product_id,quantity,amount,checkout,datetime
0,2027-T0105C01-100089,5379,1.0,1.71,1,2020-10-15 17:53:16
1,2027-T0105C01-100089,5379,1.0,1.60,1,2020-10-15 17:53:16
2,2027-T0105C01-100089,3482,1.0,0.63,1,2020-10-15 17:53:16
3,2027-T0105C01-100089,3059,1.0,0.45,1,2020-10-15 17:53:16
4,2027-T0105C01-100089,3059,1.0,0.45,1,2020-10-15 17:53:16


## 3. Steps to follow

To put order and meaning to this project it need to be done following this steps: 

1.   **Data cleaning**: Review that the data obtained has no gabs and confim that prices and units are in confirmance ranks.
2.   **Data modeling**: Choose ML algorithm for suggestions. Making models for products and families.
3.   **Training and testing**: Define a training dataset and a Test dataset. Execute the process and test that if model generated complies.
4.   **Analyse the results**: Make some strategies to analyse if the results given are as expected and detect if some parameter affects negatively to results. Some kind of plot can be a helper to visualize the results.



# 4. Data Exploration

https://colab.research.google.com/drive/14cBbxQoxdX_3kSPNl9m9djfAu3raOLfm?usp=sharing


#5. Methodology

In our project, we chose to try different recommendation algorithms, so we can compare the results and the accuracy of each one. At the end, we will be able to choose the most suitable for us.
To compare the performance, we decided to use a benchmarck algorithm : it will recommend randomly a product among the top 10 best-selling products of the store (https://colab.research.google.com/drive/1eZv1LIOMXJC2oiDcBY-4iInMcGYUB2xl?usp=sharing)

##Evaluation Strategy 
*How will we evaluate the performance of our models and compare them?*



*   Train the model with 80% of the data. We used the data of 2020 for one store.
*   With the test dataset : for the purchases with 3 products or more, delete the first product.
*   Run the model on these purchases (that are incomplete).
*   Compare the prediction.
*   Calculate the *precision* by dividing the number of good precisions by the size of the test dataset.

In the future we will try to get the *coverage* metric, because it will allow us to check the percentage of the catalog that is recommended at least once. This is a good metric to see if the recommendation algorithm is balanced, and not recommmend the same products almost each time. 

The *recall* can be calculated when you recommend many products and you want to see the global accuracy of your recommended list. 

## The algorithms



###Frequent Pattern Mining (FP-Growth)

*   Based on user-item interaction matrix
*   The Item-based model (off-line) : we “ssume that users will mostly be interested by items similar to the one they have interacted with in the past”. Here, the similarity between two items is based on how many users had bought both in the past.


###Collaborative filtering (ALS)


*   Based on user-item interaction matrix
*   The matrix factorization approach can be seen as an extension of the item-based one, where we consider both similarity between the items and between the users.
*   The main drawback of matrix factorization is that you have to retrain from scratch your model each time there is a new user or a new user-item interaction to take it into account.


###Neural Network


*   With the neural networks model, we actually do on-line recommendations. By on-line, we mean that there is no need to precompute daily recommendations for all users — recommendations are computed on the fly, considering the latest items a user has interacted with.
*  we can’t capitalize on sequences of item interactions (we don’t know in which order the product has been put in the shopping cart) → it would have been a great advantage according to Decathlon’s team results.


#7. Obstacles


*   Choose algorithms and technologies.
*   Find the adequate performance metrics.
*   Dilemma between working with families or products.
*   Home office work that implies communication troubles.
*   Complexity of Neural Network model (lack of references).
*   We don’t know the profile of each buyer, neither the different carts they have bought.


#8. Results

Here are the precision we obtained for each model.

*   Top 10 : 0.85%
*   FP-Growth : 2.44%
*   ALS : 3.21%
*   NN : --  (MSE : 0.2803522050380707) 





#9. Conclusions


For one subject, a recommendation system, we had many options for the algorihms. To select the algorithms for recommendation system, we have researched about the history of the recommendation system. Moreover, we were able to know the difference of the Content based filtering and Collaborative Filtering method to implement the recommendation system.
So, we decided to implement both and compare them. We also tried to implement a Neural Network model.

We started by implementing a methodology to be able to compare the results and calculated recommandations with a benchmarck algorithm.

At the end, the best solution for us seems to be to use the Collaborative Filtering (ALS) method in this project, because it has better results than the Content based filtering (FP-Growth). Some other teams have used a model based on Neural Network with very good results (as the Decathlon team for example), but it is more difficult to build a model that suits in our case.

To continue this project, we plan to refine our recommendation taking into account the purchase period. It was a goal that we could not accomplish due to lack of time.



